In [ ]:
! rm -r sample_data
! pip -q install transformers
! pip -q install pytorch-lightning
! pip -q install datasets

     |████████████████████████████████| 4.9 MB 9.0 MB/s 
     |████████████████████████████████| 6.6 MB 54.1 MB/s 
     |████████████████████████████████| 120 kB 68.2 MB/s 
     |████████████████████████████████| 708 kB 8.4 MB/s 
     |████████████████████████████████| 419 kB 24.3 MB/s 
     |████████████████████████████████| 5.9 MB 22.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is incompatible.
     |████████████████████████████████| 431 kB 7.0 MB/s 
     |████████████████████████████████| 115 kB 73.8 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 127 kB 69.3 MB/s 


In [ ]:
import os
import math
import pandas as pd
import numpy as np
from pprint import pprint
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
from datasets import load_dataset
from pytorch_lightning import seed_everything
import pickle as pkl
from scipy.stats import spearmanr, pearsonr
os.chdir("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation")
RANDOM_SEED = 42
seed_everything(RANDOM_SEED)
device = "cuda" if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.seed:Global seed set to 42


In [ ]:
def process_coral_context(context):
    context = context.rstrip("[SEP]")
    context = context.replace("[CLS]", "Speaker 1:")
    speaker_num = 2
    while context.find("[SEP]") != -1:
        context = context.replace("[SEP]", f"Speaker {speaker_num}:", 1)
        if speaker_num == 2:
            speaker_num = 1
        else:
            speaker_num = 2
    return "Review this dialog: Context: " + context

In [ ]:
def process_blenderbot_context(context):
    return "Review this dialog: Context: Speaker 1: " + context

# **Create Dataset**

In [ ]:
def process_data(model_feedback_annotations_sheet):
    processed_datapoints = []
    for i in range(len(model_feedback_annotations_sheet)):
        dataset_here = model_feedback_annotations_sheet.at[i, "dataset"]
        model_here = model_feedback_annotations_sheet.at[i, "model_name"]
        # if dataset_here.startswith("Dialo"):
        # print(dataset_here)
        if type(model_feedback_annotations_sheet.at[i, 'formatted_context']) == str:
            # print(dataset_here)
            context_here = model_feedback_annotations_sheet.at[i, "formatted_context"]
            context_here = process_coral_context(str(context_here).replace("nan", ""))

        elif model_here == "Blenderbot-1B":
            context_here = model_feedback_annotations_sheet.at[i, 'context']
            context_here = process_blenderbot_context(str(context_here).replace("nan", ""))
        
        else:
            if model_here == "DialoGPT":
                # print("yeah")
                context_here = "[CLS] " + model_feedback_annotations_sheet.at[i, 'context'].replace("<|endoftext|>", " [SEP] ") + " [SEP]"
            else:    
                # print(dataset_here)
                context_here = model_feedback_annotations_sheet.at[i, 'context']
            context_here = process_coral_context(context_here)
            
        # if dataset_here.startswith("Blender"):
        #     context_here = process_blenderbot_context(context_here)
        # else:
        #     context_here = process_coral_context(context_here)
    
        model_response_here = model_feedback_annotations_sheet.at[i, 'model_response']
        human_feedback_here = model_feedback_annotations_sheet.at[i, 'human_feedback']
        if type(model_response_here) == float:
            model_response_here = ""
        datapoint = context_here + " Response: " + str(model_response_here.replace("nan", "")) + " Feedback: " + human_feedback_here
        processed_datapoints.append(datapoint)
    model_feedback_annotations_sheet["processed_datapoint"] = processed_datapoints
    return model_feedback_annotations_sheet

In [ ]:
feedback_annotations_filepath = "/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/annotated-feedback-model-responses/iitkgp-mtp-dialog-model-responses-1.csv"
model_feedback_annotations_sheet = pd.read_csv(feedback_annotations_filepath)

In [ ]:
print(list(model_feedback_annotations_sheet['dataset']))

['dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'dailydialog', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversations', 'Reddit r/CasualConversation

In [ ]:
model_feedback_annotations_sheet

In [ ]:
gpt2_large_checkpoint = "gpt2-large"
gpt2_tokenizer = AutoTokenizer.from_pretrained(gpt2_large_checkpoint,
                                               pad_token="<|pad|>",
                                               additional_special_tokens=["<|startofcontext|>",
                                                                          "<|endofcontext|>",
                                                                          "<|startofresponse|>",
                                                                          "<|endofresponse|>"]
                                               )
gpt2_model = AutoModelForCausalLM.from_pretrained(gpt2_large_checkpoint).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# i = 47
# list(model_feedback_annotations_sheet['dataset'])

In [ ]:
model_feedback_annotations_sheet_processed = process_data(model_feedback_annotations_sheet)

In [ ]:
model_feedback_annotations_sheet_processed

# **Scratch**

In [ ]:
model_feedback_annotations_sheet_processed.to_csv("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/annotated-feedback-model-responses/annotated-feedback-processed.csv",
                                                  index=False)

In [ ]:
dataset = load_dataset("csv", data_files={"train": ["annotated-feedback-processed.csv"]})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-0dc349f645b7170f/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['model_name', 'Unnamed: 1', 'dataset', 'context', 'formatted_context', 'model_response', 'human_feedback', 'Unnamed: 7', 'questions', 'filename', 'processed_datapoint'],
        num_rows: 37
    })
})

In [ ]:
def tokenize_function(examples):
    return gpt2_tokenizer(examples["processed_datapoint"])
dataset = dataset.map(tokenize_function, batched=True, num_proc=4, remove_columns=["model_name",
                                                                                   "Unnamed: 1",
                                                                                   "dataset",
                                                                                   "context",
                                                                                   "formatted_context",
                                                                                   "model_response",
                                                                                   "human_feedback",
                                                                                   "Unnamed: 7",
                                                                                   "questions",
                                                                                   "filename",
                                                                                    ])

#0:   0%|          | 0/1 [00:00<?, ?ba/s]

#1:   0%|          | 0/1 [00:00<?, ?ba/s]

#2:   0%|          | 0/1 [00:00<?, ?ba/s]

#3:   0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
training_args = TrainingArguments(
    "gpt2-large",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    # push_to_hub=True,
)

In [ ]:
gpt2_tokenizer.pad_token="<|pad|>"

In [ ]:
gpt2_model.resize_token_embeddings(len(gpt2_tokenizer))

Embedding(50262, 1280)

In [ ]:
trainer = Trainer(
    model=gpt2_model,
    tokenizer=gpt2_tokenizer,
    args=training_args,
    train_dataset=dataset["train"],
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: processed_datapoint. If processed_datapoint are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 37
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 15


ValueError: ignored

In [ ]:
len(dataset['train']['input_ids']

37

In [ ]:
gpt2_tokenizer("Hey there how are you right now?")

{'input_ids': [10814, 612, 703, 389, 345, 826, 783, 30], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
! pip -q install transformers

In [ ]:
!nvidia-smi

Tue Aug 23 17:42:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# **Working->**

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoModel, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset, random_split
import random
from torch import cuda
import os
import numpy as np

In [ ]:
# Set the random seed to a fixed value to get reproducible results 
# torch.manual_seed(42)
# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/666 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model = GPT2LMHeadModel.from_pretrained("gpt2-large").to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))

Embedding(50259, 1280)

In [ ]:
lines = []
annotations = pd.read_csv("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/annotated-feedback-model-responses/annotated-feedback-processed.csv")
for i in range(len(annotations)):
    lines.append(annotations.at[i, 'processed_datapoint'])

In [ ]:
max_length = 256
descriptions = [description for description in lines if len(tokenizer.encode(description)) < max_length-2]
max_length = max([len(tokenizer.encode(description)) for description in descriptions])
max_length

In [ ]:
class NetflixDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer("<|startoftext|>" 
                                        + txt +    
                                        "<|endoftext|>",
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]

dataset = NetflixDataset(descriptions, tokenizer, max_length)

In [ ]:
train_size = int(1 * len(dataset))
train_dataset, val_dataset = random_split(dataset, 
                            [train_size, len(dataset) - train_size])
# train_dataset = dataset
train_size = len(train_dataset)

In [ ]:
lr=0.5e-5
bs=10
ne=3
training_args = TrainingArguments(output_dir=f'gpt2-large-lm-lr{lr}-bs{bs}-ne{ne}',
                                  overwrite_output_dir=True,
                                  num_train_epochs=3,
                                  learning_rate=2e-5,
                                  save_strategy='epoch',
                                #   logging_steps=200,
                                #   save_steps =1000,
                                  per_device_train_batch_size=1,
                                  gradient_accumulation_steps=10,
                                #   per_device_eval_batch_size=2,
                                #   warmup_steps=500,
                                  weight_decay=0.01,  
                                  load_best_model_at_end=True,
                                  )
                                #   logging_dir='./logs')

In [ ]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=train_dataset,
                #   eval_dataset=val_dataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])}
                 )

In [ ]:
# Start training process!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 70
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 10
  Total optimization steps = 21


Step,Training Loss


Saving model checkpoint to gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-7
Configuration saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-7/config.json
Model weights saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-7/pytorch_model.bin
Saving model checkpoint to gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-14
Configuration saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-14/config.json
Model weights saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-14/pytorch_model.bin
Saving model checkpoint to gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-21
Configuration saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-21/config.json
Model weights saved in gpt2-large-lm-lr5e-06-bs10-ne3/checkpoint-21/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=21, training_loss=1.6736891610281808, metrics={'train_runtime': 227.0799, 'train_samples_per_second': 0.925, 'train_steps_per_second': 0.092, 'total_flos': 191010387456000.0, 'train_loss': 1.6736891610281808, 'epoch': 3.0})

In [ ]:
model.eval()

In [ ]:
lines

['Review this dialog: Context: Speaker 1: hey man, you wanna buy some weed? Speaker 2: some what?  Response: i like to look at all my wedding, and i\'d like a little tired. Feedback: The response is incorrect because it is completely unrelated to the context. A better response would have been "Weed - marijuana. Would you like some?"',
 'Review this dialog: Context: Speaker 1: we\'ve managed to reduce our energy consumption in our factory by about 15 per cent in the last two years. Speaker 2: that\'s excellent. how have you managed that?  Response: i don\'t know, so that\'s right? Feedback: The response is incorrect because it is not coherent. "so that\'s right?" following "i don\'t know" does not make sense in the given context. A better response would be "mainly because we\'ve invested in a heat recovery system."',
 'Review this dialog: Context: Speaker 1: we\'ve managed to reduce our energy consumption in our factory by about 15 per cent in the last two years. Speaker 2: that\'s exce

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2-large",    
                            bos_token="<|startoftext|>",
                            eos_token="<|endoftext|>",
                            pad_token="<|pad|>")
checkpoint = "/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/gpt2-large-lm-lr2e-5-bs10-ne-3/checkpoint-7"
model = GPT2LMHeadModel.from_pretrained(checkpoint).to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))
model.eval()

In [ ]:
# prompt = """Review this dialog: Context: Oh , thank you . Could I interest you in our store credit card ? Response: Yes, I'd like to purchase some groceries. Feedback: """
# tokenized = tokenizer(prompt, return_tensors="pt")
# input_ids = tokenized['input_ids'].to('cuda')
# # attn_mask = tokenized['attention_mask'].to('cuda')

# # generate up to 30 tokens
# outputs = model.generate(input_ids, do_sample=False, max_length=200)
# tokenizer.batch_decode(outputs, skip_special_tokens=True)
# # ['Today I believe we can finally get to the point where we can make a difference in the lives of the people of the United States of America.\n']

In [ ]:
test_data = pd.read_csv("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/annotated-feedback-model-responses/iitkgp-mtp-dialog-model-responses-2.csv")
# test_data

In [ ]:
# for i in range(len(test_data)):
#     if test_data.at[i, 'model_name'] == "CORAL ESIM-gen":
#         print(process_coral_context(test_data.at[i, 'context']))

In [ ]:
def generate_results(model, tokenizer, test_data, dialog_model_name, learning_rate, batch_size, num_epochs):
    model.config.pad_token_id = tokenizer.pad_token_id
    print(f"Feedback results for {dialog_model_name}'s responses, with\nModel Learning_rate = {learning_rate}, Batch_size = {batch_size}, Num_epochs = {num_epochs}:\n")
    for i in range(len(test_data)):
        if test_data.at[i, 'model_name'] == dialog_model_name:
            if dialog_model_name.startswith("Blender"):
                prompt = process_blenderbot_context(test_data.at[i, 'context']) + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            elif dialog_model_name.startswith("CORAL"):
                prompt = process_coral_context(test_data.at[i, 'context']) + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            tokenized = tokenizer(prompt, return_tensors="pt")
            input_ids = tokenized['input_ids'].to('cuda')

            outputs = model.generate(input_ids, do_sample=False, max_length=200, num_beams=5)
            print("Prompt: " + str(tokenizer.decode(input_ids[0])))
            model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            feedback_starts = model_response.find("Feedback: ")
            print("Model Response: " + model_response[feedback_starts + 11:])
            print()

In [ ]:
## checkpoint-21 (3 epochs)
generate_results(model, tokenizer, test_data, "Blenderbot-1B", "5e-6", "10", "3")

Feedback results for Blenderbot-1B's responses, with
Model Learning_rate = 5e-6, Batch_size = 10, Num_epochs = 3:

Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "I love being a dog."

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: Well, I love to travel. I've been all over the world. What about you?

Prompt: Review this dialog: Context: Speaker 1: DAMN YOU PIZZA Response:  I know, right?  I was so mad at myself.  I should have known better. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "I know, right?"

Prompt: Review this dialog: Context: Spe

In [ ]:
## checkpoint-21 (3 epochs)
generate_results(model, tokenizer, test_data, "Blenderbot-1B", "5e-6", "10", "3")

In [ ]:
## checkpoint-7 (1 epoch)
generate_results(model, tokenizer, test_data, "Blenderbot-1B", "2e-5", "10", "1")

Feedback results for Blenderbot-1B's responses, with
Model Learning_rate = 1e-5, Batch_size = 10, Num_epochs = 1:

Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work, but it's worth it. Response: Yes, I love being a dog. It's a lot of work

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: Well, I l

In [ ]:
## checkpoint-14 (2 epochs)
generate_results(model, tokenizer, test_data, "Blenderbot-1B", "1e-5", "10", "1")

Feedback results for Blenderbot-1B's responses, with
Model Learning_rate = 1e-5, Batch_size = 10, Num_epochs = 1:

Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "Yes, I love being a dog."

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "What about you?"

Prompt: Review this dialog: Context: Speaker 1: DAMN YOU PIZZA Response:  I know, right?  I was so mad at myself.  I should have known better. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "I know, rig

In [ ]:
## checkpoint-21 (3 epochs)
generate_results(model, tokenizer, test_data, "Blenderbot-1B", "1e-5", "10", "1")

Feedback results for Blenderbot-1B's responses, with
Model Learning_rate = 1e-5, Batch_size = 10, Num_epochs = 1:

Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "Yes, I love being a dog."

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: The response is incorrect because it is unrelated to the context. A better response would be, "What about you?"

Prompt: Review this dialog: Context: Speaker 1: DAMN YOU PIZZA Response:  I know, right?  I was so mad at myself.  I should have known better. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "I know, ri

In [ ]:
### 1 epoch, lr=2e-5, bs=10
### CORAL ESIM-gen
model.config.pad_token_id = tokenizer.pad_token_id
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "CORAL ESIM-gen":
        # print('yay')
        # try:
            prompt = process_coral_context(test_data.at[i, 'context']) + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            tokenized = tokenizer(prompt, return_tensors="pt")
            input_ids = tokenized['input_ids'].to('cuda')
            # attn_mask = tokenized['attention_mask'].to('cuda')

            # generate up to 30 tokens
            outputs = model.generate(input_ids, do_sample=False, max_length=200, num_beams=5)
            print("Prompt: " + str(tokenizer.decode(input_ids[0])))
            print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].replace(prompt, ""))
            print()
        # except:
        #     continue

In [ ]:
### 1 epoch, lr=2e-5, bs=1
### CORAL ESIM-gen
model.config.pad_token_id = tokenizer.pad_token_id
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "CORAL ESIM-gen":
        # print('yay')
        # try:
            prompt = process_coral_context(test_data.at[i, 'context']) + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            tokenized = tokenizer(prompt, return_tensors="pt")
            input_ids = tokenized['input_ids'].to('cuda')
            # attn_mask = tokenized['attention_mask'].to('cuda')

            # generate up to 30 tokens
            outputs = model.generate(input_ids, do_sample=False, max_length=200, num_beams=5)
            print("Prompt: " + str(tokenizer.decode(input_ids[0])))
            print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].replace(prompt, ""))
            print()
        # except:
        #     continue

Prompt: Review this dialog: Context: Speaker 1: i really need to start eating healthier. Speaker 2: i have to start eating better too.  Response: how long is it? how did you know????'s?? Feedback: 
Model Response:  The response is incorrect because it is grammatically incorrect. A better response would be, "How long is it?"

Prompt: Review this dialog: Context: Speaker 1: good afternoon! can i help you? Speaker 2: could you show me where the chinesc - style clothing is located? i want to buy a silk coat. Speaker 1: this way, please. here they are. they're all handmade.  Response: that's right. i'm so happy to see you tomorrow..?. Feedback: 
Model Response: 

Prompt: Review this dialog: Context: Speaker 1: next, please. hello, may i help you, sir? Speaker 2: yes, i want to send a registered airmail letter to france.  Response: i just went out. that's why i was my out. Feedback: 
Model Response:  The response is incorrect because it is grammatically incorrect. A better response would be,

In [ ]:
### 1 epoch, lr=2e-5
### BLENDERBOT
model.config.pad_token_id = tokenizer.pad_token_id
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "Blenderbot-1B":
            prompt = process_blenderbot_context(test_data.at[i, 'context']) + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            tokenized = tokenizer(prompt, return_tensors="pt")
            input_ids = tokenized['input_ids'].to('cuda')

            outputs = model.generate(input_ids, do_sample=False, max_length=200, num_beams=5)
            print("Prompt: " + str(tokenizer.decode(input_ids[0])))
            model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
            feedback_starts = model_response.find("Feedback: ")
            print("Model Response: " + model_response[feedback_starts + 11:])
            print()

Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "Yes, I love being a dog."

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: The response is incorrect because it does not answer the question. A better response would be, "I love to travel."

Prompt: Review this dialog: Context: Speaker 1: DAMN YOU PIZZA Response:  I know, right?  I was so mad at myself.  I should have known better. Feedback: 
Model Response: The response is incorrect because it is grammatically incorrect. A better response would be, "I know, right?"

Prompt: Review this dialog: Context: Speaker 1: That's great! I never learned :/ Response:  Neither did I

In [ ]:
### 2 epoch, lr=0.5e-5
### BLENDERBOT
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "Blenderbot-1B":
        prompt = "Review this dialog: Context: Speaker 1: " + test_data.at[i, 'context'] + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
        tokenized = tokenizer(prompt, return_tensors="pt")
        input_ids = tokenized['input_ids'].to('cuda')
        # attn_mask = tokenized['attention_mask'].to('cuda')

        # generate up to 30 tokens
        outputs = model.generate(input_ids, do_sample=False, max_length=200)
        print("Prompt: " + str(tokenizer.decode(input_ids[0])))
        print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
        print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "I love being a dog."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback:  The response is incorrect because it is not a question. A better response would be, "Tell me about yourself."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback:  The response is incorrect because it is not relevant to the context. A better response would be, "I have never heard of Tsingtao beer."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "I love that show, too."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "I'm looking for a flight to Beijing. What's the best way to get there?"

Prompt: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character created by J.K. Rowling. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional c

In [ ]:
### 1 epoch, lr=2e-5
### BLENDERBOT
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "Blenderbot-1B":
        try:
            prompt = "Review this dialog: Context: Speaker 1: " + test_data.at[i, 'context'] + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
            tokenized = tokenizer(prompt, return_tensors="pt")
            input_ids = tokenized['input_ids'].to('cuda')
            # attn_mask = tokenized['attention_mask'].to('cuda')

            # generate up to 30 tokens
            outputs = model.generate(input_ids, do_sample=False, max_length=200)
            print("Prompt: " + str(tokenizer.decode(input_ids[0])))
            print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
            print()
        except:
            continue

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: For what? Are you taking up painting? No. I can't paint. I'm asking for my niece. She is really into it. Response: <no-response> Feedback: 
Model Response: Review this dialog: Context: Speaker 1: For what? Are you taking up painting? No. I can't paint. I'm asking for my niece. She is really into it. Response: <no-response> Feedback:  Response: I'm sorry, I'm not sure what you mean. I'm not sure if I can paint. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything. I'm not sure if I can do anything



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Oh, thank you. Could I interest you in our store credit card? Response: Yes, I'd like to purchase some groceries. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Oh, thank you. Could I interest you in our store credit card? Response: Yes, I'd like to purchase some groceries. Feedback:  Response: Thank you for your interest in our store credit card. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We are happy to help you. We



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: Review this dialog: Context: Speaker 1: Honey, I need to have a talk with you. Dad, I have to do my homework. Response: You're not going to do your homework? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Honey, I need to have a talk with you. Dad, I have to do my homework. Response: You're not going to do your homework? Feedback:  Response: I'm not going to do my homework. Feedback: The response is incorrect because it is grammatically incorrect. A better response would be, "I'm not going to do my homework."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: What's wrong with you, young man? Doctor, I have a bad cough and a headache. Response: Oh, I'm sorry to hear that. What's the matter? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: What's wrong with you, young man? Doctor, I have a bad cough and a headache. Response: Oh, I'm sorry to hear that. What's the matter? Feedback:  Response: I'm sorry to hear that. What's the matter?



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Don't worry, we'll soon fix them. Good, I hope you'll make sure that it won't happen again. Response: <no-response> Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Don't worry, we'll soon fix them. Good, I hope you'll make sure that it won't happen again. Response: <no-response> Feedback:  Response: I'm sorry, I don't know what to say. I'm sorry, I don't know what to say. Feedback: The response is incorrect because it is not a response. A better response would be, "I'm sorry, I don't know what to say."

Prompt: Review this dialog: Context: Speaker 1: Of course. It sounds great. I'll take it. Response: Alright, I'll take it. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Of course. It sounds great. I'll take it. Response: Alright, I'll take it. Feedback:  Response: I'll take it. 



In [ ]:
### 1 epoch, lr=2e-5
### BLENDERBOT
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "Blenderbot-1B":
        prompt = "Review this dialog: Context: Speaker 1: " + test_data.at[i, 'context'] + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
        tokenized = tokenizer(prompt, return_tensors="pt")
        input_ids = tokenized['input_ids'].to('cuda')
        # attn_mask = tokenized['attention_mask'].to('cuda')

        # generate up to 30 tokens
        outputs = model.generate(input_ids, do_sample=False, max_length=200)
        print("Prompt: " + str(tokenizer.decode(input_ids[0])))
        print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
        print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "I love being a dog."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback:  The response is incorrect because it is not a question. A better response would be, "I love to travel. What about you?"



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback:  The response is incorrect because it is unrelated to the context. A better response would be, "I have never heard of Tsingtao beer."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "Oh, I love that show! It's one of my favorites."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback:  The response is incorrect because it is grammatically incorrect. A better response would be, "Hi, I'm a fan of your airline. What do you think of them?"

Prompt: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character created by J.K. Rowling. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character crea

In [ ]:
### 2 epochs, lr=2e-5
### BLENDERBOT
for i in range(len(test_data)):
    if test_data.at[i, 'model_name'] == "Blenderbot-1B":
        prompt = "Review this dialog: Context: Speaker 1: " + test_data.at[i, 'context'] + " Response: " + test_data.at[i, 'model_response'] + " Feedback: "
        tokenized = tokenizer(prompt, return_tensors="pt")
        input_ids = tokenized['input_ids'].to('cuda')
        # attn_mask = tokenized['attention_mask'].to('cuda')

        # generate up to 30 tokens
        outputs = model.generate(input_ids, do_sample=False, max_length=200)
        print("Prompt: " + str(tokenizer.decode(input_ids[0])))
        print("Model Response: " + tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])
        print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I love being a dog."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Well, I love to travel. I've been all over the world."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Prompt: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I have never heard of Tsingtao beer. What is it?"



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Oh, I love that show! It's one of my favorites."



The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Sure, what do you want to know about them?"

Prompt: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character created by J.K. Rowling. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional char

In [ ]:
text = """
Prompt: Review this dialog: Context: Speaker 1: And fun to be aound. Hahha Response: Yes, I love being a dog.  It's a lot of work, but it's worth it. Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I love being a dog."

Prompt: Review this dialog: Context: Speaker 1: Tell me 3 interesting facts about yourself. Response:  Well, I love to travel. I've been all over the world. What about you? Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Well, I love to travel. I've been all over the world."

Prompt: Review this dialog: Context: Speaker 1: DAMN YOU PIZZA Response:  I know, right?  I was so mad at myself.  I should have known better. Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I know, right? I was so mad at myself."

Prompt: Review this dialog: Context: Speaker 1: That's great! I never learned :/ Response:  Neither did I, but I'm glad I did. It was a lot of fun. Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "That's great! I never learned about it."

Prompt: Review this dialog: Context: Speaker 1: I just passed my General Chemistry exam! :) Response:  Congratulations!  That's a big accomplishment.  What are you going to do with it? Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Congratulations! That's a big accomplishment. What are you going to do with it?"

Prompt: Review this dialog: Context: Speaker 1: I've been in a movie, a tv show, and a music video. Response:  That's pretty cool. What kind of music do you like to listen to while you're in the movie? Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I like to listen to a lot of different kinds of music."

Prompt: Review this dialog: Context: Speaker 1: Just found out my fourth consecutive crush is gay. Again. Response:  Oh no, I'm sorry to hear that. Have you tried talking to them about it? Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Oh no, I'm sorry to hear that. Have you tried talking to them about it?"

Prompt: Review this dialog: Context: Speaker 1: 😀😊☺🍔🍟 Response:  Hi, how are you today? I'm doing well, thanks for asking. How are you? Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I'm doing well, thanks for asking."

Prompt: Review this dialog: Context: Speaker 1: I was wearing a "Hakuna Matata" Lion King shirt yesterday and an African man walked up to me and pointed at it asking "Do you know what that means?" Response:  Oh my gosh, that is so creepy! Did you say anything to him? I would have been so scared! Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Oh, I'm sure you know what that means. What's the story behind it?"

Prompt: Review this dialog: Context: Speaker 1: if you could be anything.. Response:  I'm not sure what you mean by that, but thank you for the compliment. Feedback: 
Model Response:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I'm not sure what you mean by that, but thank you for the compliment."

Prompt: Review this dialog: Context: Speaker 1: Welcome to IBA. Can I help you?  Response:  Sure, what do you know about IBA?  I have been studying it for a few years now. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Welcome to IBA. Can I help you?  Response:  Sure, what do you know about IBA?  I have been studying it for a few years now. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Sure, what do you know about IBA?"

Prompt: Review this dialog: Context: Speaker 1: Hello, this is Lucy. May I speak to Mr. Smith?  Response:  Hi Lucy, how are you today? I am fine, thank you for asking. How are you? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Hello, this is Lucy. May I speak to Mr. Smith?  Response:  Hi Lucy, how are you today? I am fine, thank you for asking. How are you? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Hi, how are you doing?"

Prompt: Review this dialog: Context: Speaker 1: Waiter, we have been waiting for 20 minutes. Would you mind hurrying? Response:  I don't mind at all. I wait tables, so I know how you feel. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Waiter, we have been waiting for 20 minutes. Would you mind hurrying? Response:  I don't mind at all. I wait tables, so I know how you feel. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I don't mind at all. I wait tables, so I know how you feel."

Prompt: Review this dialog: Context: Speaker 1: The train is leaving. Hurry up! Which car are we in?  Response:  It's a Toyota Corolla.  It's the best car I've ever owned. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: The train is leaving. Hurry up! Which car are we in?  Response:  It's a Toyota Corolla.  It's the best car I've ever owned. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "It's a Toyota Corolla. What car are you in?"

Prompt: Review this dialog: Context: Speaker 1: The consignments of chemical instruments have just reached us. We have to lodge a claim against you for inferior quality I am afraid. Response:  Oh no, I'm sorry to hear that. What kind of instruments do you play? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: The consignments of chemical instruments have just reached us. We have to lodge a claim against you for inferior quality I am afraid. Response:  Oh no, I'm sorry to hear that. What kind of instruments do you play? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Oh no, I'm sorry to hear that. What kind of instruments do you play?"

Prompt: Review this dialog: Context: Speaker 1: Good morning.sir.My name is Liu Peng from Tsinghai University. I am here for the interview relating to my visa application. Response:  That's great! I hope you get the job. What kind of job are you applying for? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Good morning.sir.My name is Liu Peng from Tsinghai University. I am here for the interview relating to my visa application. Response:  That's great! I hope you get the job. What kind of job are you applying for? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "That's great! I hope you get the job. What kind of job are you applying for?"

Prompt: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: May I recommend you Tsingtao beer?  Response:  I have never heard of Tsingo beer.  What is it?  Do you like it? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "I have never heard of Tsingtao beer. What is it?"

Prompt: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Marry Lin's Office, this is Linda.  Response:  Oh, I love that show!  It's one of my favorites.  Have you seen it? Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Oh, I love that show! It's one of my favorites."

Prompt: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Hello, China Southern Airlines. Can I help you?  Response:  Sure, what do you want to know about them? They are an American multinational airline based in Honolulu, Hawaii. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Sure, what do you want to know about them?"

Prompt: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character created by J.K. Rowling. Feedback: 
Model Response: Review this dialog: Context: Speaker 1: Peter, enough with your computer games. Go do your homework now.  Response:  Peter Parker is a fictional character created by J.K. Rowling. Feedback:  The response is incorrect because it is grammatically wrong and unrelated to the context. A better response would be, "Peter, enough with your computer games. Go do your homework now."

"""

# **Likert Prompts Training**

In [ ]:
prefix = """Human Evaluation of Chatbot Outputs:

Annotation Instructions: 
You will be given a conversation between two individuals. You will then be given a potential chatbot-generated response for the next turn in the conversation. Your task is to rate the response on several metrics. The response for one metric should not influence other metrics. For example, if a response is not understandable or has grammatical errors - you should try to ignore this when considering whether it maintains context or if it is interesting.
The following are the metrics and corresponding rating scales that each response is required to be rated on:
Understandable (0 - 1): Is the response understandable in the context of the history? A score of 0 (no) means that the response is difficult to understand. You do not know what the person is trying to say. A score of 1 (yes) means that the response is understandable. You know what the person is trying to say.
Natural (1 - 3): Is the response naturally written? A score of 1 (bad) means that the response is unnatural. A score of 2 (ok) means the response is strange, but not entirely unnatural. A response of 3 (good) means that the response is natural.
Maintains Context (1 - 3): Does the response serve as a valid continuation of the conversation history? A score of 1 (no) means that the response drastically changes topic or ignores the conversation history. A score of 2 (somewhat) means the response refers to the conversation history in a limited capacity (e.g.,in a generic way) and shifts the conversation topic. A score of 3 (yes) means the response is on topic and strongly acknowledges the conversation history.
Interesting (1 - 3): Is the response dull or interesting? A score of 1 (dull) means that the response is generic and dull. A score of 2 (somewhat interesting) means the response is somewhat interesting and could engage you in the conversation (e.g., an opinion, thought). A score of 3 (interesting) means the response is very interesting or presents an interesting fact.
Uses Knowledge (0 - 1): Given the fact that the response is conditioned on, how well does the response use that fact? A score of 0 (no) means the response does not mention or refer to the fact at all. Ascore of 1 (yes) means the response uses the fact well.
Overall Quality (1 - 5): Given your answers above, what is your overall impression of the generated response/utterance? A score of 1 (very bad) means the response is completely invalid, and it would be difficult to recover the conversation after this. A score of 2 (bad) means that the response is valid, but otherwise poor in quality. A score of 3 (neutral) means the response is neither good nor bad, and has no negative qualities, but no positive ones either. A score of 4 (good) means this is a good response, but falls short of being perfect because of a key flaw. A score of 5 (very good) means this response is good and does not have any strong flaws."""

In [ ]:
with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/tc_all_prompts.pkl", "rb") as f1:
    tc_all_prompts = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/tc_all_labels.pkl", "rb") as f1:
    tc_all_labels = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/pc_all_prompts.pkl", "rb") as f1:
    pc_all_prompts = pkl.load(f1)

with open("/content/drive/MyDrive/iitkgp-mtp-dialog-response-generation/pc_all_labels.pkl", "rb") as f1:
    pc_all_labels = pkl.load(f1)

pc_all_prompts_labelled = [-1] * len(pc_all_prompts)
tc_all_prompts_labelled = [-1] * len(tc_all_prompts)
for i, each in enumerate(pc_all_prompts):
    prompt_here = pc_all_prompts[i]
    label_1 = round(np.mean(np.array(pc_all_labels[i][0])))
    label_2 = round(np.mean(np.array(pc_all_labels[i][1])))
    label_3 = round(np.mean(np.array(pc_all_labels[i][2])))
    label_4 = round(np.mean(np.array(pc_all_labels[i][3])))
    label_5 = round(np.mean(np.array(pc_all_labels[i][4])))
    label_6 = round(np.mean(np.array(pc_all_labels[i][5])))
    # all_prompts[i] = prefix + each + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Overall Quality: {str(label_5)}"
    # pc_all_prompts_labelled[i] = each.lstrip() + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Uses Knowledge: {str(label_5)}\n6. Overall Quality: {str(label_6)}"
    pc_all_prompts_labelled[i] = each.lstrip() + f"\n1. {str(label_1)}\n2. {str(label_2)}\n3. {str(label_3)}\n4. {str(label_4)}\n5. {str(label_5)}\n6. {str(label_6)}"

for i, each in enumerate(tc_all_prompts):
    prompt_here = tc_all_prompts[i]
    label_1 = round(np.mean(np.array(tc_all_labels[i][0])))
    label_2 = round(np.mean(np.array(tc_all_labels[i][1])))
    label_3 = round(np.mean(np.array(tc_all_labels[i][2])))
    label_4 = round(np.mean(np.array(tc_all_labels[i][3])))
    label_5 = round(np.mean(np.array(tc_all_labels[i][4])))
    label_6 = round(np.mean(np.array(tc_all_labels[i][5])))
    # all_prompts[i] = prefix + each + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Overall Quality: {str(label_5)}"
    # tc_all_prompts_labelled[i] = each.lstrip() + f"\n1. Understandable: {str(label_1)}\n2. Natural: {str(label_2)}\n3. Maintains Context: {str(label_3)}\n4. Interesting: {str(label_4)}\n5. Uses Knowledge: {str(label_5)}\n6. Overall Quality: {str(label_6)}"
    tc_all_prompts_labelled[i] = each.lstrip() + f"\n1. {str(label_1)}\n2. {str(label_2)}\n3. {str(label_3)}\n4. {str(label_4)}\n5. {str(label_5)}\n6. {str(label_6)}"

In [ ]:
print(pc_all_prompts_labelled[0])

Context:
Person 1: lead singer for a band , music teacher
Person 2: wow nice are you really good ?
Person 1: millions of plays on soundcloud
Person 2: really would you share or are you shy

Facts:
Person 1's statement: i also have a dog walking business.
Person 1's statement: i've three dogs.
Person 1's statement: my father was a door to door salesman.
Person 1's statement: i am in an open polyamorous relationship.
Person 1's statement: i like to watch the olympics.

Generated response: 
Person 1: ha ha i'm so shy

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a person would naturally say?
3. Maintains Context (1 - 3): Does the response serve as a valid continuation of the preceding conversation?
4. Interesting (1 - 3): Is the response dull or interesting?
5. Uses Knowledge (0 - 1): Given the facts that the response is conditioned on, how well d

In [ ]:
import torch
from transformers import GPT2Tokenizer, GPTNeoForCausalLM, GPTNeoModel, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import pandas as pd
from torch.utils.data import Dataset, random_split
import random
from torch import cuda
import os
import numpy as np

In [ ]:
# Set the random seed to a fixed value to get reproducible results 
# torch.manual_seed(42)
# Download the pre-trained GPT-Neo model's tokenizer
# Add the custom tokens denoting the beginning and the end 
# of the sequence and a special token for padding

# model_checkpoint = "gpt2-medium"
# model_checkpoint = "gpt2-large"
model_checkpoint = "bigscience/bloom-560m"
# model_checkpoint = "bigscience/bloom-1b1"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# tokenizer = GPT2Tokenizer.from_pretrained(model_checkpoint,
#                             bos_token="<|startoftext|>",
#                             eos_token="<|endoftext|>",
#                             pad_token="<|pad|>")

Downloading:   0%|          | 0.00/222 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
# model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

Downloading:   0%|          | 0.00/710 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

In [ ]:
tc_lines = tc_all_prompts_labelled
pc_lines = pc_all_prompts_labelled
max_length = 1024
tc_descriptions = [description for description in tc_lines if len(tokenizer.encode(description)) < max_length-2]
pc_descriptions = [description for description in pc_lines if len(tokenizer.encode(description)) < max_length-2]
pc_max_length = max([len(tokenizer.encode(description)) for description in pc_descriptions])
tc_max_length = max([len(tokenizer.encode(description)) for description in tc_descriptions])
print(pc_max_length)
print(len(pc_descriptions))
print(tc_max_length)
print(len(tc_descriptions))

371
300
823
360


In [ ]:
class PromptsDataset(Dataset):
    def __init__(self, txt_list, tokenizer, max_length):
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        for txt in txt_list:
            # Encode the descriptions using the GPT-Neo tokenizer
            encodings_dict = tokenizer("<|startoftext|>" 
                                        + txt +    
                                        "<|endoftext|>",
                                        truncation=True,
                                        max_length=max_length, 
                                        padding='max_length')
            input_ids = torch.tensor(encodings_dict['input_ids'])    
            self.input_ids.append(input_ids)
            mask = torch.tensor(encodings_dict['attention_mask'])
            self.attn_masks.append(mask)
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx]


tc_dataset = PromptsDataset(tc_descriptions, tokenizer, max_length)
pc_dataset = PromptsDataset(pc_descriptions, tokenizer, max_length)

In [ ]:
pc_train_size = int(0.7 * len(pc_dataset))
tc_train_size = int(0.7 * len(tc_dataset))

pc_train_dataset, pc_trainval_dataset = random_split(pc_dataset, 
                            [pc_train_size, len(pc_dataset) - pc_train_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

pc_test_size = int(0.5 * len(pc_trainval_dataset))
pc_test_dataset, pc_val_dataset = random_split(pc_trainval_dataset, 
                            [pc_test_size, len(pc_trainval_dataset) - pc_test_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_train_dataset, tc_trainval_dataset = random_split(tc_dataset, 
                            [tc_train_size, len(tc_dataset) - tc_train_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))
tc_test_size = int(0.5 * len(tc_trainval_dataset))
tc_test_dataset, tc_val_dataset = random_split(tc_trainval_dataset, 
                            [tc_test_size, len(tc_trainval_dataset) - tc_test_size],
                            generator=torch.Generator().manual_seed(RANDOM_SEED))

# test_size = int(0.5 * len(trainval_dataset))
# val_dataset, test_dataset = random_split(trainval_dataset, [test_size, len(trainval_dataset) - test_size])
# train_dataset = dataset

In [ ]:
_, pc_unlabelled_trainval = random_split(pc_all_prompts,
                                         [pc_train_size, len(pc_dataset) - pc_train_size],
                                         generator=torch.Generator().manual_seed(RANDOM_SEED))
pc_unlabelled_test, pc_unlabelled_val = random_split(pc_unlabelled_trainval,
                                                     [pc_test_size, len(pc_trainval_dataset) - pc_test_size],
                                                     generator=torch.Generator().manual_seed(RANDOM_SEED))

_, tc_unlabelled_trainval = random_split(tc_all_prompts, 
                                         [tc_train_size, len(tc_dataset) - tc_train_size],
                                         generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_unlabelled_test, tc_unlabelled_val = random_split(tc_unlabelled_trainval, 
                                                     [tc_test_size, len(tc_trainval_dataset) - tc_test_size],
                                                     generator=torch.Generator().manual_seed(RANDOM_SEED))

In [ ]:
print(tokenizer.decode(tc_train_dataset[0][0], skip_special_tokens=True))

In [ ]:
print(tc_val_labels[4])

In [ ]:
pc_train_labels, pc_trainval_labels = random_split(pc_all_labels, 
                                              [pc_train_size, len(pc_dataset) - pc_train_size],
                                              generator=torch.Generator().manual_seed(RANDOM_SEED))
pc_test_labels, pc_val_labels = random_split(pc_trainval_labels,
                                             [pc_test_size, len(pc_trainval_labels) - pc_test_size],
                                             generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_train_labels, tc_trainval_labels = random_split(tc_all_labels, 
                                              [tc_train_size, len(tc_dataset) - tc_train_size],
                                              generator=torch.Generator().manual_seed(RANDOM_SEED))

tc_test_labels, tc_val_labels = random_split(tc_trainval_labels,
                                             [tc_test_size, len(tc_trainval_labels) - tc_test_size],
                                             generator=torch.Generator().manual_seed(RANDOM_SEED))

In [ ]:
print(tc_val_labels[0])

In [ ]:
print(tokenizer.decode(pc_train_dataset[0][0], skip_special_tokens=True))

In [ ]:
len(tc_train_dataset)

251

In [ ]:
print(len(pc_train_dataset))
print(len(tc_train_dataset))

In [ ]:
print(tokenizer.decode(tc_pc_concatdataset[251][0], skip_special_tokens=True))

In [ ]:
tc_pc_concatdataset = torch.utils.data.ConcatDataset([tc_train_dataset, pc_train_dataset])

In [ ]:
lr=0.5e-5
bs=10
ne=4
training_args = TrainingArguments(output_dir=f'bloom-tc-medium-lm-lr{lr}-bs{bs}-ne{ne}',
                                  overwrite_output_dir=True,
                                  num_train_epochs=ne,
                                  learning_rate=lr,
                                  save_strategy='epoch',
                                  evaluation_strategy='epoch',
                                #   logging_steps=200,
                                #   save_steps =1000,
                                  per_device_train_batch_size=1,
                                  gradient_accumulation_steps=bs,
                                #   per_device_eval_batch_size=2,
                                #   warmup_steps=500,
                                  weight_decay=0.01,  
                                  load_best_model_at_end=True,
                                  fp16=True,
                                  )
                                #   logging_dir='./logs')

In [ ]:
trainer = Trainer(model=model, args=training_args,  
                  train_dataset=tc_pc_concatdataset,
                  eval_dataset=tc_val_dataset, 
                  # This custom collate function is necessary 
                  # to built batches of data
                  data_collator=lambda data: 
              {'input_ids': torch.stack([f[0] for f in data]),       
               'attention_mask': torch.stack([f[1] for f in data]),
               'labels': torch.stack([f[0] for f in data])}
                 )

Using cuda_amp half precision backend


In [ ]:
# Start training process!
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 461
  Num Epochs = 4
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 10
  Gradient Accumulation steps = 10
  Total optimization steps = 184


Epoch,Training Loss,Validation Loss


RuntimeError: ignored

In [ ]:
model_checkpoint = f'gpt2-tc-medium-lm-lr{lr}-bs{bs}-ne{ne}/checkpoint-46'
model = GPT2LMHeadModel.from_pretrained(model_checkpoint).to(device)
# model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-1.3B").cuda()
# Resize the token embeddings because we've just added 3 new tokens 
model.resize_token_embeddings(len(tokenizer))
model.config.pad_token_id = tokenizer.pad_token_id

loading configuration file gpt2-tc-medium-lm-lr1e-05-bs10-ne4/checkpoint-46/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2-medium",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 1024,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 24,
  "n_positions": 1024,
  "n_special": 0,
  "pad_token_id": 50258,
  "predict_special_tokens": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "to

In [ ]:
each = """Context:
Person 1:  the guy is a machine! but he needs a better stage name. maybe president banana? 
Person 2:  i do not think the president of zimbabwe would be happy about that 
Person 1:  he could be the artist formerly known as president banana? i don't really care as long as i can listen to him on the radio! i love two things on my radio : indonesian pop music and electromagnetic storms from jupiter. those are my go to listening pleasures! 
Person 2:  yeah that is so cool that if you turn your radio to am, you may capture jupiter's storms 

Facts:
according to canadian law, all radios are required to have at least 40 % of the music played be canadian.

Generated response: 
Person 1: that might be banned in canada. the law says that all stations must have 40 % of the music played be canadian!

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a person would naturally say?
3. Maintains Context (1 - 3): Does the response serve as a valid continuation of the preceding conversation?
4. Interesting (1 - 3): Is the response dull or interesting?
5. Uses Knowledge (0 - 1): Given the facts that the response is conditioned on, how well does the response use the facts?
6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?

Answers:
"""
tokenized = tokenizer(each)
input_ids = torch.tensor([tokenized.input_ids]).to(device)
if len(input_ids[0]) < 1000:
    generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=5)
    print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

Context:
Person 1:  the guy is a machine! but he needs a better stage name. maybe president banana? 
Person 2:  i do not think the president of zimbabwe would be happy about that 
Person 1:  he could be the artist formerly known as president banana? i don't really care as long as i can listen to him on the radio! i love two things on my radio : indonesian pop music and electromagnetic storms from jupiter. those are my go to listening pleasures! 
Person 2:  yeah that is so cool that if you turn your radio to am, you may capture jupiter's storms 

Facts:
according to canadian law, all radios are required to have at least 40 % of the music played be canadian.

Generated response: 
Person 1: that might be banned in canada. the law says that all stations must have 40 % of the music played be canadian!

Questions about the generated response:
1. Understandable (0 - 1): Is the response understandable given the previous context?
2. Natural (1 - 3): Does the response seem like something that a 

In [ ]:
tokenizer.decode(tc_train_dataset[0][0], skip_special_tokens=True)[-200:]

'does the response use the facts?\n6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?\n\nAnswers:\n1. 1\n2. 2\n3. 2\n4. 3\n5. 1\n6. 4'

In [ ]:
tc_unlabelled_test[0][-200:]

'e is conditioned on, how well does the response use the facts?\n6. Overall Quality (1 - 5): Given your answers above, what is your overall impression of the quality of the generated response?\n\nAnswers:'

In [ ]:
tc_test_dataset = tc_unlabelled_test
for i, each in enumerate(tc_test_dataset):
    # print(each)
    tokenized = tokenizer(each + "\n")
    input_ids = torch.tensor([tokenized.input_ids]).to(device)
    input_text = tokenizer.decode(input_ids[0], skip_special_tokens=True)
    if len(input_ids[0]) < 1000:
        generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=3, temperature=0.1)
        print(f"\nnum = {i}")
        print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0].replace(input_text, ""))


num = 0
1.
2.
3.

num = 1
1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.

num = 2
1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.

num = 3
1.
2.
3.

num = 4
1.
2.
3.
4.


KeyboardInterrupt: ignored

In [ ]:
str1 = "hey there"
tokenized = tokenizer(str1)
input_ids = torch.tensor([tokenized.input_ids]).to(device)
generation = model.generate(input_ids, do_sample=False, max_new_tokens=100, num_beams=5)
print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


hey theres


In [ ]:
input_ids = torch.tensor([test_dataset[0][0].tolist()]).to(device)
if len(input_ids[0]) < 1000:
    generation = model.generate(input_ids, do_sample=False, max_new_tokens=24, num_beams=5)
    print(tokenizer.batch_decode(generation, skip_special_tokens=True)[0])

In [ ]:
print(tokenizer.decode(test_dataset[0][0]))

<|startoftext|> Human Evaluation of Chatbot Outputs:

Annotation Instructions: 
You will be given a conversation between two individuals. You will then be given a potential chatbot-generated response for the next turn in the conversation. Your task is to rate the response on several metrics. The response for one metric should not influence other metrics. For example, if a response is not understandable or has grammatical errors - you should try to ignore this when considering whether it maintains context or if it is interesting.
The following are the metrics and corresponding rating scales that each response is required to be rated on:
Understandable (0 - 1): Is the response understandable in the context of the history? A score of 0 (no) means that the response is difficult to understand. You do not know what the person is trying to say. A score of 1 (yes) means that the response is understandable. You know what the person is trying to say.
Natural (1 - 3): Is the response naturally wr

In [ ]:
for each in test_dataset:
    input_ids = torch.tensor([each[0].tolist()]).to('cuda')
    if len(input_ids[0]) < 1000:
        outputs = model.generate(input_ids, do_sample=False, max_new_tokens=24, num_beams=5)
        print(str(tokenizer.decode(input_ids[0])))
        model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        print(model_response)
        print("\n")
    else:
        print(len(input_ids[0]))

1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018
1018


In [ ]:
tokenized['input_ids']

tensor([[20342,   612]])

tensor([[50257, 20490, 34959,  ..., 50258, 50258, 50258]], device='cuda:0')

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id
# prompt = "hey there"
# tokenized = tokenizer(prompt, return_tensors="pt")
# input_ids = tokenized['input_ids'].to('cuda')

outputs = model.generate(torch.tensor([test_dataset[0][0].tolist()]).to('cuda'), do_sample=False, max_new_tokens=100, num_beams=5)
print("Prompt: " + str(tokenizer.decode(input_ids[0])))
model_response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
# feedback_starts = model_response.find("Feedback: ")
print("Model Response: " + model_response)
# print()

RuntimeError: ignored